# Real roots

In [1]:
using JuMP, DynamicPolynomials, MomentTools

In [2]:
X = @polyvar x
p = x^4-1

x⁴ - 1

We minimize nothing under the constraint p1=0, that is, we search the real roots of p. 

In [3]:
using MosekTools; optimizer = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true)
v, M = minimize(nothing, [p], [], X, 4,optimizer)

(0.0, 
A MOMent program with:
A JuMP Model
Feasibility problem with:
Variables: 9
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 6 constraints
`Array{GenericAffExpr{Float64,VariableRef},1}`-in-`MathOptInterface.PositiveSemidefiniteConeSquare`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Mosek
Names registered in the model: basis, degree, index, moments, monomials, nu, variables, y)

We use Mosek optimizer to solve the underlying SDP optimization problem. The first argument of the function `minimize` is the objective function. Here it is `nothing` that is $0$. The second argument is the array of equality constraints, the third is the array of non-negativity constraints (here empty).

The function `minimize` returns `v` the optimal value found (here it is $0$ since we optimize "$0$") and `M` the Moment Model built for the optimization (which type is `MomentTools.MOM.Model`). 

The minimizers can be obtained from `M` as follows:

In [4]:
getminimizers(M)

1×2 Array{Float64,2}:
 -1.0  1.0

The minimizers are presented as a $n\times r$ matrix, where $n$ is the number of variables and $r$ is the number of points.

Next, we search the root(s) with $x \geq 0$.

In [5]:
v, M = minimize(nothing, [p], [x], X, 4,optimizer)
getminimizers(M)

1×1 Array{Float64,2}:
 1.0000000000000004

Now we search the root with minimal value of $x$.

In [6]:
v, M = minimize(x, [p], [], X, 4,optimizer)
getminimizers(M)

1×1 Array{Float64,2}:
 -0.9999999999999999